<h1><center style="color:#2B3698">Merging the different methods for generating matching keywords</center></h1>

## Table Of Contents :
1. [Synset selection for each category](#syn)
2. [Wordnet Scores](#score-wordnet)
3. [Vocabulary Generator](#vocab)
4. [Putting All Together](#all)
5. [Next Steps](#next)

#### Importing the different libraries

In [60]:
# Import libraries 
from nltk.corpus import wordnet as wn
from stemming.porter2 import stem
from nltk.corpus import wordnet_ic
from tqdm import tqdm
from time import time
import gensim
import os
import pandas as pd

# *Vocabulary creation* 

The process that we will use in order to generate the different words will be the following :
<ol> 
<li> Define different models that can generate n words with a score (wordnet, word2vec).
   <br>The output of this generators will be of the form {word : {"model" : score}}.
<li> Given the outputs of different models, merge them into one dictionnary and define 
   the weights for each model.
<li> Loop through all the categories and generate all the words.

Defining global variables :

In [26]:
# Path of the pretrained word2vec 
PATH = r""
# Data for wordnet score
BROWN_IC = wordnet_ic.ic('ic-brown.dat')

# Categories used in weanswer
CATEGORIES = [
    'advice',
    'hygiene',
    'equipment',
    'activities',
    'technology',
    'info',
    'administrative',
    'job',
    'education',
    'home',
    'health',
    'food'
]

## 1. Synset selection for each category <a id = "syn">

The goal here is to choose the right synset for each category.

In [27]:
# For each category, for each synset print its lemma names and the definition
for category in CATEGORIES:
    print(">>> {}".format(category))
    i = 0
    for synset in wn.synsets(category):
        print(i, "-", synset.lemma_names(), ":", synset.definition())
        i += 1
    print("\n")

>>> advice
(0, '-', [u'advice'], ':', u'a proposal for an appropriate course of action')


>>> hygiene
(0, '-', [u'hygiene'], ':', u'a condition promoting sanitary practices')
(1, '-', [u'hygiene', u'hygienics'], ':', u'the science concerned with the prevention of illness and maintenance of health')


>>> equipment
(0, '-', [u'equipment'], ':', u'an instrumentality needed for an undertaking or to perform a service')


>>> activities
(0, '-', [u'activity'], ':', u'any specific behavior')
(1, '-', [u'action', u'activity', u'activeness'], ':', u'the state of being active')
(2, '-', [u'bodily_process', u'body_process', u'bodily_function', u'activity'], ':', u'an organic process that takes place in the body')
(3, '-', [u'activity'], ':', u'(chemistry) the capacity of a substance to take part in a chemical reaction')
(4, '-', [u'natural_process', u'natural_action', u'action', u'activity'], ':', u'a process existing in or produced by nature (rather than by the intent of human beings)')
(5, '-

In [28]:
# For each category, we will select only the relevant synsets (the numbers represent the indices of the right synsets)
d_category_synsets = {"advice": [0], "hygiene": [0], "equipment": [0], "activities": [0], "technology": [0], "info": [0], 
                      "administrative": [0], "job": [0, 1], "education": [0], "home": [0], "health": [0], "food": [0,1]}

## 2. Wordnet scores   <a id="score-wordnet"> 

Here we compute 4 different scores for wordnet (http://www.nltk.org/howto/wordnet.html):
<li> path_similarity score :  Return a score denoting how similar two word senses are, based on the shortest path that connects the senses in the is-a (hypernym/hypnoym) taxonomy. The score is in the range 0 to 1.
<li> lch_similarity : Return a score denoting how similar two word senses are, based on the shortest path that connects the senses (as above) and the maximum depth of the taxonomy in which the senses occur. The relationship is given as -log(p/2d) where p is the shortest path length and d the taxonomy depth. (Here is a website that simulates the path to better understand the taxonomy : http://ws4jdemo.appspot.com/?mode=w&s1=&w1=cat&s2=&w2=dog.
<li> wup_similarity : The Wu & Palmer measure (wup) calculates similarity by considering the depths of the two concepts in wordnet, along with the depth of the LCS (least common ancestor in the taxonomy) The formula is score = 2*depth(lcs) / (depth(s1) + depth(s2)). This means that 0 $<$ score $<=$ 1. The score can never be zero because the depth of the LCS is never zero (the depth of the root of a taxonomy is one). The score is one if the two input concepts are the same.
<li> jcn_similarity : Jiang-Conrath Similarity Return a score denoting how similar two word senses are, based on the Information Content (IC) of the Least Common Subsumer (most specific ancestor node) and that of the two input Synsets. The relationship is given by the equation 1 / (IC(s1) + IC(s2) - 2 * IC(lcs)). (Information Content (IC) is a measure of specificity for a concept. Higher values are associated with more specific concepts (e.g., pitch fork), while those with lower values are more general (e.g., idea). In- formation Content is computed based on frequency counts of concepts as found in a corpus of text).

Also, this article http://ftp.cs.toronto.edu/pub/gh/Budanitsky+Hirst-2001.pdf presents the different score computation and compares them to human judgment of semantic similarity. The similarity that matched the most the human judgment was jcn_similarity.

In [29]:
def score_wordnet(word, matching_keyword) : 
    """
    Function that generates a score for the wordnet outputs
    
    Parameters
    ----------
    word              : string we want to compute the similarity to.
    matching_keywords : string for computring the similarity.
    
    Returns
    -------
    score : The similarity score (float).
    """
    
    # We will nomalise the scores for each similarity
    min_max_values = {'path' : [0, 1], 'lch' : [0, 3.6375861597263857 ], 'wup' : [0,1], 'jcn' : [0, 10000]}
    # Reference for range : (lch) https://stackoverflow.com/questions/20112828/maximum-score-in-wordnet-based-similarity
    # (jcn) https://stackoverflow.com/questions/35751207/how-to-normalize-similarity-measurements-lch-wup-path-res-lin-jcn-between
    
    # For jcn we will take 10 000 as the maximum (quite arbitrary but seemed relevant)
    
    word = wn.synsets(word)[0]
    matching_keyword = wn.synsets(matching_keyword)[0]
    score = 0
    try :
        score += word.path_similarity(matching_keyword)
    except :
        pass
    try : 
        score += (word.lch_similarity(matching_keyword) /  min_max_values['lch'][1])
    except :
        pass
    try :
        score += word.wup_similarity(matching_keyword)
    except :
        pass
    try : 

        score += (word.jcn_similarity(matching_keyword) / min_max_values['jcn'][1])
    except :
        pass
    
    return score

## 3. Vocabulary generator  <a id="vocab">

Here we create a class that is able to generate different words according to different models.

We will keep only stems in our inputs and then generate the lems afterward.

In [36]:
class VocabularyGenerator():
    
    def __init__(self):
        pass
    
    def generate_words_wordnet(self, word, n, depth = 5, synsets_indices = d_category_synsets) :
        """
        Function that generates matching keywords given a word.

        Parameters
        ----------
        word    : word to compute similarities to (can be a category word).
        depth   : number of layers we use when generating maching keywords.
        n       : number of words we take.
        synsets : dictionary mapping a word to the a list of indices
                 specifying its relevant synsets in wordnet
                 Ex: {"food": [0]}

        Returns
        -------
        d : {similar_word: {wordnet: wordnet_score}
        """

        name_model = "wordnet"
        d = {}
        previous_keys = []

        # First iteration (select only relevant synsets as specified in d_category_synsets)
        # All the synsets of word
        synsets = wn.synsets(word)    
        # Only relevant synsets of word
        if word in CATEGORIES:
            synsets = [synsets[i] for i in range(len(synsets)) if i in synsets_indices[word]]
        for synset in tqdm(synsets):
            for lemma in synset.lemma_names():
                d[stem(lemma)] = {name_model : score_wordnet(word, lemma)}

        # Other iterations

        for i in tqdm(range(depth)) :
            previous_keys = [key for key in d.keys() if key not in previous_keys]
            for origin in previous_keys:
                for synset in wn.synsets(origin):
                    for lemma in synset.lemma_names():
                        d[stem(lemma)] = {name_model : score_wordnet(word, lemma)}

        if n > len(d) :
            return d

        d_max_values = {k: d[k] for k in sorted(d, key = lambda k: d[k]["wordnet"], reverse = True)[:n]}

        return d_max_values

    def generate_words_word2vec(self, word, n, path = PATH, model_name = "text8-vector.bin"):
        """ 
        Most similar words to word and their scores.

        Parameters
        ----------
        word       : word to compute similarities to (can be a category word)
        n          : number of similar words to get
        path       : path of the pretrained word2vec model
        model_name : name of the pretrained word2vec model

        Returns
        -------
        d : {similar_word: {word2vec: word2vec_score}
        """
        
        # Load Google's pre-trained Word2Vec model
        model = gensim.models.KeyedVectors.load_word2vec_format(os.path.join(path, "text8-vector.bin"), binary=True) 
        d = {key: {"word2vec": value} for (key, value) in model.most_similar(word, topn = n)}
        return d
    
    def generate_word_spacy(self):
        pass


In [37]:
# Examples
generator = VocabularyGenerator()
d_wordnet = generator.generate_words_wordnet("food", 40)
d_word2vec = generator.generate_words_word2vec("food", 40)

100%|██████████| 4/4 [00:00<00:00, 15.20it/s]


In [38]:
d_wordnet

{u'aliment': {'wordnet': 2.218539496664136},
 u'back': {'wordnet': 0.9169821263560435},
 u'backup': {'wordnet': 0.9169821263560435},
 u'baffl': {'wordnet': 0.7746942172313404},
 u'beat': {'wordnet': 0.8404107581326701},
 u'begin': {'wordnet': 0.7746942172313404},
 u'brook': {'wordnet': 0.9169821263560435},
 u'cargo_area': {'wordnet': 0.7174237954982008},
 u'cargo_deck': {'wordnet': 0.7174237954982008},
 u'come': {'wordnet': 1.107911697275812},
 u'comest': {'wordnet': 2.218539496664136},
 u'conserv': {'wordnet': 1.3407650777506728},
 u'donjon': {'wordnet': 0.7174237954982008},
 u'draw': {'wordnet': 0.7746942172313404},
 u'eatabl': {'wordnet': 2.218539496664136},
 u'edibl': {'wordnet': 2.218539496664136},
 u'food': {'wordnet': 3.0},
 u'gravel': {'wordnet': 1.107911697275812},
 u'guard': {'wordnet': 0.8404107581326701},
 u'handgrip': {'wordnet': 0.9169821263560435},
 u'handl': {'wordnet': 0.9169821263560435},
 u'harbor': {'wordnet': 0.7746942172313404},
 u'harbour': {'wordnet': 0.77469421

In [39]:
d_word2vec

{u'bananas': {'word2vec': 0.6068102717399597},
 u'beef': {'word2vec': 0.6227272748947144},
 u'beverages': {'word2vec': 0.6333991289138794},
 u'canned': {'word2vec': 0.614071249961853},
 u'cashew': {'word2vec': 0.6168663501739502},
 u'chemicals': {'word2vec': 0.5928492546081543},
 u'citrus': {'word2vec': 0.5871732234954834},
 u'cocoa': {'word2vec': 0.6041184663772583},
 u'cooking': {'word2vec': 0.6009650826454163},
 u'dairy': {'word2vec': 0.6460014581680298},
 u'fermented': {'word2vec': 0.5760336518287659},
 u'fertilizers': {'word2vec': 0.5925314426422119},
 u'fish': {'word2vec': 0.5927667021751404},
 u'foods': {'word2vec': 0.7001853585243225},
 u'foodstuffs': {'word2vec': 0.6066914200782776},
 u'fruit': {'word2vec': 0.5887770652770996},
 u'fruits': {'word2vec': 0.6141229867935181},
 u'grain': {'word2vec': 0.5781475305557251},
 u'ingredients': {'word2vec': 0.5744912028312683},
 u'liquor': {'word2vec': 0.5889161825180054},
 u'livestock': {'word2vec': 0.6447887420654297},
 u'maize': {'wor

## 4. Putting it all together <a id="all">

First we create a function that normalizes the scores of a model.

In [40]:
def normalize_scores(d_scores):
    """"
    The function normalize the scores (between 0 and 1).
    
    Parameters
    ----------
    d_scores : dictionnary of the form {matching_keywords : {model : score}}.
    
    Returns
    -------
    d_normalized : same dictionnary with normalized scores.
    """
    
    # Get the model name
    modelname = list(list(d_scores.values())[0].keys())[0]
        
    scores = [d_scores[k][modelname] for k in d_scores.keys()]
    M = max(scores)
    m = min(scores)
    
    # Normalize the scores
    # If there is only one word, thus one score in the generated vocabulary
    if m == M:
        d_normalized = {k: {modelname: d_scores[k][modelname] / M} for k in d_scores.keys()}
    else:
        d_normalized = {k: {modelname: (d_scores[k][modelname] - m) / (M - m)} for k in d_scores.keys()}
    
    return d_normalized

Merging all the result obtained :

In [41]:
def mixing_model(models, word, n, generator = VocabularyGenerator()):
    """
    Function that takes as input a list of models and a word and outputs
    a normalized dictionary that gathers all the scores of the different
    models.
    
    Parameters
    ----------
    models    : list of models (names).
    word      : the word in question.
    n         : input to the generators (number of words).
    generator : default value, our created classe.
    
    Returns
    ------- 
    d_score, d_count : output dictionnary with the generate words 
                       (scores and counts).
    
    """
    
    d = {}
    for modelname in models :
        d_scores = getattr(generator, "generate_words_{}".format(modelname))(word, n)
        d.update(normalize_scores(d_scores))
        
    d_count = {}
    d_score = {}
    # counting occurrence (could also use counter)
    for key in d.keys() :
        d_count[key] = len(d[key])
        d_score[key] = sum(d[key].values())
        
    return d_score, d_count    

In [42]:
mixing_model(["word2vec", "wordnet"], "food", n = 40)

100%|██████████| 4/4 [00:00<00:00, 16.44it/s]


({u'aliment': 0.6576410015163425,
  u'back': 0.08742679892319248,
  u'backup': 0.08742679892319248,
  u'baffl': 0.025090256185177198,
  u'bananas': 0.2628202745200088,
  u'beat': 0.05388076962859285,
  u'beef': 0.3884821873510358,
  u'begin': 0.025090256185177198,
  u'beverages': 0.4727345793662483,
  u'brook': 0.08742679892319248,
  u'canned': 0.32014440793358967,
  u'cargo_area': 0.0,
  u'cargo_deck': 0.0,
  u'cashew': 0.34221122766036843,
  u'chemicals': 0.15260052393050463,
  u'citrus': 0.10778926643611968,
  u'cocoa': 0.2415689492826425,
  u'come': 0.17107332539762463,
  u'comest': 0.6576410015163425,
  u'conserv': 0.2730867346391723,
  u'cooking': 0.21667354480349313,
  u'dairy': 0.5722277304830711,
  u'donjon': 0.0,
  u'draw': 0.025090256185177198,
  u'eatabl': 0.6576410015163425,
  u'edibl': 0.6576410015163425,
  u'fermented': 0.019844326666481577,
  u'fertilizers': 0.1500914549071075,
  u'fish': 0.15194878713429213,
  u'food': 1.0,
  u'foods': 1.0,
  u'foodstuffs': 0.261881961

Function that generates the final output :

In [52]:
def categories_generate(models, n, categories = CATEGORIES, d = ({}, {})) :
    """
    Function that generate the final output.
    
    Parameters
    ----------
    models     : list of models (names).
    n          : input to the generators (number of words).
    categories : list of categories (default value, the global variable
                 CATEGORIES).
    dics        : already generated dictionaries to complete 
                 (d_output_score, d_output_count)
                 
    Returns
    -------
    d_output_score, d_output_count : output dictionnary with the generate words 
                       (scores and counts) for each category.
     
    """
    
    d_output_score = d[0]
    d_output_count = d[1]
    for word in tqdm(categories):
        if word in d_output_score.keys():
            continue
        print(">>> Generating vocabulary for {}".format(word))
        d_output_score[word], d_output_count[word] = mixing_model(models, word, n)
        # Select only the top n similar words
        d_output_score[word] = {k: d_output_score[word][k] for k in sorted(d_output_score[word], 
                                key = lambda k: d_output_score[word][k], reverse = True)[:n]}
        d_output_count[word] = {k: d_output_count[word][k] for k in d_output_score[word].keys()}

        
    return d_output_score, d_output_count

In [56]:
d_output_score

{'activities': {u'actions': 0.33122927078220643,
  u'activ': 1.0,
  u'agencies': 0.6445560650020895,
  u'agency': 0.184608955222044,
  u'amway': 0.011512767037084834,
  u'aspects': 0.018355148921207785,
  u'business': 0.1474387557985668,
  u'centers': 0.02971471797343061,
  u'centres': 0.04353376069828749,
  u'decentralize': 0.21662841075086856,
  u'efforts': 0.10558063651301698,
  u'esro': 0.043154457253839984,
  u'facilities': 0.0636962151638011,
  u'financial': 0.004455200433261129,
  u'funding': 0.08232130631737115,
  u'governmental': 0.1414794157127298,
  u'hospitals': 0.16228757815058348,
  u'humanitarian': 0.16681799325713528,
  u'icrc': 0.05535692284927948,
  u'institutions': 0.24759255365505628,
  u'interests': 0.06791362033140293,
  u'issues': 0.24532165501198128,
  u'matters': 0.1452767569278636,
  u'msf': 0.3031935781127454,
  u'neap': 0.3122456409712937,
  u'ngos': 0.42907171711407605,
  u'nongovernmental': 0.2391274421120034,
  u'operations': 0.17006468308089762,
  u'orga

In [53]:
d_output_score, _ = categories_generate(["word2vec", "wordnet"], 40)

  0%|          | 0/12 [00:00<?, ?it/s]

>>> Generating vocabulary for advice



  8%|▊         | 1/12 [00:01<00:13,  1.26s/it]s]

>>> Generating vocabulary for hygiene



 17%|█▋        | 2/12 [00:02<00:12,  1.24s/it]s]

...


 83%|████████▎ | 10/12 [00:53<00:18,  9.32s/it]

>>> Generating vocabulary for health



 92%|█████████▏| 11/12 [01:02<00:08,  8.97s/it]

>>> Generating vocabulary for food



100%|██████████| 12/12 [01:03<00:00,  6.73s/it]


Getting better formatted output :

In [61]:
def render_output(d) :
    """
    Function that renders a re-formated input (dataframe)
    that could be easily written in a file.
    
    Parameters
    ----------
    d : output dictionary from our model.
    
    Returns
    -------
    data : multi index dataframe, reformated version.
    
    """
    
    d_generation = {}
    
    for (key, value) in d.iteritems() :
        for (key2, value2) in value.iteritems() :
            d_temp = {"score" : value2} 
            d_generation[(key, key2)] = d_temp
            
    data = pd.DataFrame(d_generation).transpose()
    data.index = pd.MultiIndex.from_tuples(data.index)
    data.index.names = ["category", "matching_keyword"]
    
    return data
    
data_output = render_output(d_output_score)
data_output.head()

score
category   matching_keyword          
activities actions           0.331229
           activ             1.000000
           agencies          0.644556
           agency            0.184609
           amway             0.011513

In [62]:
data_output

score
category   matching_keyword            
activities actions             0.331229
           activ               1.000000
           agencies            0.644556
           agency              0.184609
           amway               0.011513
           aspects             0.018355
           business            0.147439
           centers             0.029715
           centres             0.043534
           decentralize        0.216628
           efforts             0.105581
           esro                0.043154
           facilities          0.063696
           financial           0.004455
           funding             0.082321
           governmental        0.141479
           hospitals           0.162288
           humanitarian        0.166818
           icrc                0.055357
           institutions        0.247593
           interests           0.067914
           issues              0.245322
           matters             0.145277
           msf                 0.303194
           neap                0.312246
           ngos                0.429072
           nongovernmental     0.239127
           operations          0.170065
           organisation        0.109957
           organisations       0.560680
...                                 ...
technology development         0.032170
           devices             0.027454
           digital             0.210756
           electronics         0.265841
           engin               1.000000
           engineering         0.634895
           equipment           0.133219
           hardware            0.229443
           ict                 0.131760
           ieee                0.053726
           iit                 0.146209
           information         0.065867
           institute           0.448234
           kanpur              0.275518
           laboratories        0.323057
           mainframe           0.092075
           management          0.155042
           marketing           0.087323
           microfluidics       0.423738
           mos                 0.124386
           multilevel          0.009480
           networking          0.028962
           processing          0.060921
           research            0.373026
           technical           0.109713
           technolog           1.000000
           technological       0.149734
           technologies        1.000000
           telecommunications  0.229409
           uwb                 0.065939

[480 rows x 1 columns]

## 5. Next Steps <a id="next">

**The next steps for the project would be : **
<ul>
<li> Generate the lemmas from the stems for the output. 
<li> Creating other models *(spacy, word2vec with other training set, mixing word2vec with wordnet with scores as the number of occurence of each word, learn a model from the users (for instance if given a same keyword from different questions, the user answer with food related ressources 2 times in a row, then we may attribute this keyword to food and with a score of 2)* ....)
<li> Defining a relevant threshold in order to extract the most relevant words.
<li> Find a way to combine smartly the output of the different models for instance if model 1 generates fruit and model 2 generates fruit, it should be counted as 1 word with 2 occurences.
<li> Finally, may be we will need manual data cleaning.


##### Documentation (error)

import error : https://stackoverflow.com/questions/15526996/ipython-notebook-locale-error